In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
# df = pd.read_parquet("hf://datasets/shahules786/PoetryFoundationData/data/train-00000-of-00001-486832872ed96d17.parquet")
# print(f"\n\n======\n\n")

# print(df.columns, len(df))
# newyork = df[df['author'].isin(["John Ashbery", "Barbara Guest", "James Schuyler", "Kenneth Koch", "Frank O'Hara"])]
# shake = df[df['author'] == 'William Shakespeare']

# print(f"Shakespeare: {len(shake)} examples\nNew Yorkers: {len(newyork)} examples")
# print(f"Shakespeare avg length: {np.average([len(poem) for poem in shake['content']])}\nNew Yorkers avg length: {np.average([len(poem) for poem in newyork['content']])}")

In [ ]:
# def process_poem(poem) :
#   out = re.sub(r'[\r\n]+', ' ', poem)
#   out = re.sub(r'[.?!]+', '.', out)
#   out = re.sub(r'\s+', ' ', out)
#   out = out.lower()
#   sentence_list = out.split('.')
#   sentence_list = [sentence for sentence in sentence_list if len(sentence) > 0]
#   for i in range(len(sentence_list)) :
#      sentence_list[i] = re.sub(r'[^a-zA-Z ]', '', sentence_list[i])
#   return [[word for word in sentence.split(' ') if word != ''] for sentence in sentence_list ]

# newyork_processed = [] # [process_poem_into_list_of_words(newyork['content'].iloc[i]) for i in range(len(newyork))]
# for i in range(len(newyork)) :
#    newyork_processed += process_poem(newyork['content'].iloc[i])
# newyork_labels = [0 for i in range(len(newyork_processed))]
# shake_processed = [] # [process_poem_into_list_of_words(shake['content'].iloc[i]) for i in range(len(shake))]
# for i in range(len(shake)) :
#    shake_processed += process_poem(shake['content'].iloc[i])
# shake_labels = [1 for i in range(len(shake_processed))]
# processed_poems = newyork_processed + shake_processed

In [ ]:
# processed_poems[1]

In [ ]:
!pip install datasets

from datasets import load_dataset

ds = load_dataset("shahules786/PoetryFoundationData")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


README.md:   0%|          | 0.00/714 [00:00<?, ?B/s]

(…)-00000-of-00001-486832872ed96d17.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/13854 [00:00<?, ? examples/s]

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir='./poetry_model',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
)

if tokenizer.pad_token is None:
  tokenizer.pad_token = tokenizer.eos_token

def preprocess_function(examples):
    return tokenizer(examples['content'], truncation=True, padding='max_length', max_length=512, return_tensors='pt')

tokenized_dataset = ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/13854 [00:00<?, ? examples/s]

In [ ]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    data_collator=data_collator
)

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
500,3.622100
1000,3.504100
1500,3.468800
2000,3.402500
2500,3.396300
3000,3.393900
3500,3.374400
4000,3.227200
4500,3.267100
5000,3.259800


TrainOutput(global_step=10392, training_loss=3.290140588068797, metrics={'train_runtime': 1591.9644, 'train_samples_per_second': 26.107, 'train_steps_per_second': 6.528, 'total_flos': 1.0859819433984e+16, 'train_loss': 3.290140588068797, 'epoch': 3.0})

In [ ]:
trainer.model

T5ForConditionalGeneration(
  (shared): Embedding(32128, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=1024, out_features=4096, bias=False)
              (wo): Linear(in_features=4096, out_features=1024, bias=False)
              (d

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: mount failed

In [ ]:
trainer.save_model('./fine_tuned_gpt2')

In [ ]:
!ls

fine_tuned_gemini  poetry_model  sample_data  wandb


In [ ]:
!zip -r fine_tuned_gpt2.zip fine_tuned_gpt2

  adding: fine_tuned_gpt2/ (stored 0%)
  adding: fine_tuned_gpt2/model.safetensors (deflated 7%)
  adding: fine_tuned_gpt2/training_args.bin (deflated 52%)
  adding: fine_tuned_gpt2/config.json (deflated 52%)
  adding: fine_tuned_gpt2/generation_config.json (deflated 24%)


In [ ]:
from google.colab import files
files.download('fine_tuned_gpt2.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from transformers import pipeline, set_seed

generator = pipeline('text2text-generation', model=trainer.model, tokenizer = tokenizer)
generator("Winter winds do shake the darling buds of may.")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
The model 'GPT2LMHeadModel' is not supported for text2text-generation. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'Qwen2AudioForConditionalGeneration', 'SeamlessM4TForTextToText', 'SeamlessM4Tv2ForTextToText', 'SwitchTransformersForCo

[{'generated_text': 'Winter winds do shake the darling buds of may. The sun is a star. The moon is a'}]

In [ ]:
from transformers import TFT5ForConditionalGeneration, AutoTokenizer

In [ ]:
model_s_to_ny = AutoModelForCausalLM.from_pretrained('fine_tuned_gpt2')
print("=================\nS to NY model summary:")

model_ny_to_s = AutoModelForCausalLM.from_pretrained('fine_tuned_gpt2')
print("=================\nNY to S model summary:")

S to NY model summary:
NY to S model summary:


In [ ]:
mainpipe = pipeline('text2text-generation', model=model_ny_to_s, tokenizer=tokenizer)
# code = "Winter winds do shake the darling buds of may."
code = "Two roads diverged in a yellow wood, And sorry I could not travel both"
mainpipe(code)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
The model 'GPT2LMHeadModel' is not supported for text2text-generation. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'Qwen2AudioForConditionalGeneration', 'SeamlessM4TForTextToText', 'SeamlessM4Tv2ForTextToText', 'SwitchTransformersForCo

[{'generated_text': 'Two roads diverged in a yellow wood, And sorry I could not travel both ways. The road'}]